In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
from collections import OrderedDict

# Model
m = gp.Model("PB")

# Create decision variables for the fractions of compounds used
x1 = m.addVar(name="comp 1")
x2 = m.addVar(name="comp 2")
x3 = m.addVar(name="comp 3")
x4 = m.addVar(name="comp 4")

# The objective is to minimize the cost
obj = 20*x1+30*x2+20*x3+30*x4
m.setObjective(obj, GRB.MINIMIZE)

# % of compounds sums up to 100%
con1 = m.addConstr(x1+x2+x3+x4 == 1, name='com_sum')

# Exactly 25% of element A
con2 = m.addConstr(30*x1+10*x2+35*x3+25*x4 == 25, name='elem_a')

# At least 35% of element B
con3 = m.addConstr(20*x1+65*x2+35*x3+40*x4 >= 35, name='elem_b')

# At least 20% of element C
con4 = m.addConstr(40*x1+15*x2+25*x3+30*x4 >= 25, name='elem_c')

# At most 25% of comp1
con5 = m.addConstr(x1 <= 0.25, name='comp_1')

# At most 30% of comp 2
con6 = m.addConstr(x2 <= 0.30, name='comp_2')

# Non-negativity constraints
con7 = m.addConstr(x1>=0)
con8 = m.addConstr(x2>=0)
con9 = m.addConstr(x3>=0)
con10 = m.addConstr(x4>=0)

# Solve
m.optimize()

# Print optimal value of the objective function
print('\nTotal cost to produce one kg of the new product: %g' % m.objVal)

# Print optimal values for the decision variables
print('\nDecision variables:')
for v in m.getVars():
    print('%s = %g' % (v.varName, v.x))

# Create table for decision variables' sensitivity analysis
decision_var = OrderedDict([
    ('Name', ['x1', 'x2', 'x3', 'x4']),
    ('Final Value', [x1.x, x2.x, x3.x, x4.x]),
    ('Reduced Cost', [x1.RC, x2.RC, x3.RC, x4.RC]),
    ('Obj Coeff', [20, 30, 20, 30]),
    ('Upper Range', [x1.SAObjUp, x2.SAObjUp, x3.SAObjUp, x4.SAObjUp]),
    ('Lower Range', [x1.SAObjLow, x2.SAObjLow, x3.SAObjLow, x4.SAObjLow])
])


# Create table for constraints' sensitivity analysis
constraint = OrderedDict([
    ('Name', ['com_sum', 'elem_a', 'elem_b', 'elem_c', 'comp_1', 'comp_2']),
    ('Shadow Price', [con1.Pi, con2.Pi, con3.Pi, con4.Pi, con5.Pi, con6.Pi]),
    ('RHS Coeff', [1, 25, 35, 25, 0.25, 0.3]),
    ('Slack', [con1.Slack, con2.Slack, con3.Slack, con4.Slack, con5.Slack, con6.Slack]),
    ('Upper Range', [con1.SARHSUp, con2.SARHSUp, con3.SARHSUp, con4.SARHSUp, con5.SARHSUp, con6.SARHSUp]),
    ('Lower Range',
     [con1.SARHSLow, con2.SARHSLow, con3.SARHSLow, con4.SARHSLow, con5.SARHSLow, con6.SARHSLow])
])

# Print sensitivity analysis tables for decision variables and constraints
print('\n')
print(pd.DataFrame.from_dict(decision_var))
print('\n')
print(pd.DataFrame.from_dict(constraint))

Academic license - for non-commercial use only - expires 2022-07-19
Using license file C:\Users\jkettune\gurobi.lic
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10 rows, 4 columns and 22 nonzeros
Model fingerprint: 0x141d213c
Coefficient statistics:
  Matrix range     [1e+00, 7e+01]
  Objective range  [2e+01, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-01, 4e+01]
Presolve removed 6 rows and 0 columns
Presolve time: 0.02s
Presolved: 4 rows, 4 columns, 16 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0000000e+01   8.500000e+00   0.000000e+00      0s
       2    2.4250000e+01   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.04 seconds
Optimal objective  2.425000000e+01

Total cost to produce one kg of the new product: 24.25

Decision variables:
comp 1 = 0.25
comp 2 = 0.3
comp 3 = 0.325
comp 4 = 0.125


